In [ ]:
def extract_json_from_script(script_content):
    # Find the JSON data within the script
    start = script_content.find("window.statsWidgetData = ")
    if start == -1:
        return None
    start += len("window.statsWidgetData = ")

    # Find the end of the JSON object
    bracket_count = 0
    end = start
    for i, char in enumerate(script_content[start:]):
        if char == "{":
            bracket_count += 1
        elif char == "}":
            bracket_count -= 1
            if bracket_count == 0:
                end = start + i + 1
                break

    if bracket_count != 0:
        return None

    json_str = script_content[start:end]

    # Parse the JSON data
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None


def fetch_and_process_data(url):
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)

    try:
        driver.get(url)

        # # Click the 'loadmore' button multiple times
        # for i in range(21):
        #     try:
        #         element = WebDriverWait(driver, 10).until(
        #             EC.presence_of_element_located((By.CLASS_NAME, 'loadmore'))
        #         )
        #         driver.execute_script("arguments[0].click();", element)
        #         time.sleep(1)  # Wait for content to load
        #     except:
        #         print(f"Stopped after {i} clicks")
        #         break

        # Extract the script content
        soup = BeautifulSoup(driver.page_source)
        # print(soup)
        script_tag = soup.find(
            "script", string=lambda text: text and "window.statsWidgetData" in text
        )

        # print(script_tag)

        # script_content = script_tag

        # json_data = extract_json_from_script(script_content)

        if script_tag:
            json_data = extract_json_from_script(script_tag.string)
            if json_data:

                return json_data

                # player_data = json_data["stats"]["data"]
                # df = pd.DataFrame(player_data)

                # # Convert 'value' and 'match_played' to numeric type
                # df["value"] = pd.to_numeric(df["value"])
                # df["match_played"] = pd.to_numeric(df["match_played"])

                # # Calculate points per match
                # df["points_per_match"] = df["value"] / df["match_played"]

                # return df

    finally:
        driver.quit()

    return None


# URL of the webpage
url = "https://www.prokabaddi.com/stats/44-22-raid-points-statistics"

# Fetch and process data
json_data = fetch_and_process_data(url)
